# test_workflow.ipynb
## Kat Nykiel, Panos Manganaris
Demonstrate

    - querying materials project for structure
    - modifying structure with pymatgen
    - submit workflow with atomate
    - parse results with jobflow

### Query Materials Project

In [ ]:
# Import libraries
from mp_api.client import MPRester

In [ ]:
# Read in new Materials Project API key
with open(os.path.expanduser('~/keys.txt'), 'r') as f:
    key = f.readlines()[0]

In [ ]:
# Query for a Ti3AlC2 MAX structure
with MPRester(key) as m:
    data = m.summary.search(material_ids=["mp-3747"])
    
# Verify structure object
struct = data[0].structure
print(struct)

## Modify structure with pymatgen
For this example, let's suppose we want to try $Mo_3AlN_2$ instead of $Ti3AlC2$

In [ ]:
map = {
    'C':'N',
    'Ti':'Mo'
}
struct.replace_species(map)
print(struct)

### Submit workflow with atomate
For this example, we'll use atomate2's built-in *double relaxation* workflow, but there are many other options such as

- static
- relax
- band structure
- elastic constants
- optical absorption

In [ ]:
# Import atomate2 libraries
from atomate2.vasp.flows.core import DoubleRelaxMaker

# Create atomate2 double relax workflow
relax_job = DoubleRelaxMaker().make(struct)

We can modify the workflow for custom INCAR, KPOINTS or POTCAR settings

In [ ]:
# Modify atomate2 workflow using powerups
from atomate2.vasp.powerups import update_user_potcar_functional, update_user_incar_settings
from jobflow.managers.fireworks import flow_to_workflow

incar_updates = {
    'ISMEAR':0,
    'SIGMA':0.1,
    'ENCUT':500,
    'NCORE':8,
    'GGA':'PE',
    'PREC':'Normal',

}

relax_job = update_user_incar_settings(relax_job, incar_updates)
relax_job = update_user_potcar_functional(relax_job, potcar_functional='PBE_52')


In [ ]:
# Submit the workflow with fireworks
from fireworks import LaunchPad

wf = flow_to_workflow(relax_job)
lpad = LaunchPad.auto_load()
# TODO: fix where launches occur
lpad.add_wf(wf)

In [ ]:
!qlaunch singleshot

## Parse results with jobflow

In [ ]:
from jobflow import SETTINGS
import pprint

store = SETTINGS.JOB_STORE
store.connect()
result = store.query_one(
    {"output.formula_pretty":"AlMo3N2"},properties=["output"]
)
pprint.pprint(result)

In [ ]:
from pymatgen.core import Structure
print(Structure.from_dict(result['output']['structure']))